In [1]:
# baseline cnn model
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from os import listdir
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# load train and test dataset
def load_dataset():
    folder = "C:/Users/Dani/data/images/"
    onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    
    train_files = []
    y_train = []
    i=0
    for _file in onlyfiles:
        train_files.append(_file)

    image_width = 1000
    image_height = 800
    ratio = 2

    image_width = int(image_width / ratio)
    image_height = int(image_height / ratio)

    channels = 3
    nb_classes = 1

    dataset = np.ndarray(shape=(len(train_files), channels, image_height, image_width),
                         dtype=np.float32)
    i = 0
    for _file in train_files:
        img = load_img(folder + "/" + _file)  # this is a PIL image
        img.thumbnail((image_width, image_height))
        # Convert to Numpy Array
        x = img_to_array(img)  
        x = x.reshape(-1, 3,400,500)
        # Normalize
#         x = (x - 128.0) / 128.0
        dataset[i] = x
        i += 1
        if i % 100 == 0:
            print("%d images to array" % i)
    print("All images to array!")

#    for i in dataset:
#        print(i.size)
    
    z_train, y_train = np.loadtxt('C:/Users/Dani/labeled.txt', delimiter=' | ', unpack = True)
    
    from sklearn.model_selection import train_test_split

    #Splitting 
    X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.4, random_state=0)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
    
    from keras.utils import to_categorical
    y_binary = to_categorical(y_train)
    
    #print(y_train)
    #print(y_test)
    #print(y_val)
    
    return X_train, y_binary, X_test, y_test

In [3]:
# test_array = np.random.rand(3,2)
# test_vector = np.random.rand(4)

# np.savez_compressed('C:/Users/Dani/data/images', a = test_array, b = test_vector)


In [4]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(3, 400, 500)))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(2, activation='softmax'))
# 	compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [5]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [6]:
# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    # save model
    model.save('final_model.h5')

In [7]:
# entry point, run the test harness
run_test_harness()

100 images to array
200 images to array
300 images to array
400 images to array
500 images to array
600 images to array
700 images to array
800 images to array
900 images to array
1000 images to array
1100 images to array
1200 images to array
1300 images to array
1400 images to array
1500 images to array
1600 images to array
1700 images to array
1800 images to array
1900 images to array
2000 images to array
2100 images to array
2200 images to array
2300 images to array
2400 images to array
2500 images to array
2600 images to array
2700 images to array
2800 images to array
2900 images to array
3000 images to array
3100 images to array
3200 images to array
3300 images to array
3400 images to array
3500 images to array
3600 images to array
3700 images to array
3800 images to array
3900 images to array
4000 images to array
4100 images to array
4200 images to array
4300 images to array
4400 images to array
4500 images to array
4600 images to array
4700 images to array
4800 images to array
4

MemoryError: 

In [18]:
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=False, target_size=(800, 1000))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(-1, 3,400,500)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

In [31]:
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('sample_image_7.png')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	digit = model.predict_classes(img)
	print(digit[0])

In [32]:
# entry point, run the example
run_example()

0
